# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 12:12PM,243871,20,8364749,"ACI Healthcare USA, Inc.",Released
1,Aug 10 2022 12:12PM,243871,20,8376112,"ACI Healthcare USA, Inc.",Released
2,Aug 10 2022 12:12PM,243871,20,8376196,"ACI Healthcare USA, Inc.",Released
3,Aug 10 2022 12:12PM,243871,20,8376197,"ACI Healthcare USA, Inc.",Released
4,Aug 10 2022 12:12PM,243871,20,8376319,"ACI Healthcare USA, Inc.",Released
5,Aug 10 2022 12:12PM,243871,20,8376320,"ACI Healthcare USA, Inc.",Released
6,Aug 10 2022 12:12PM,243871,20,8380691,"ACI Healthcare USA, Inc.",Released
7,Aug 10 2022 12:12PM,243871,20,8380692,"ACI Healthcare USA, Inc.",Released
8,Aug 10 2022 12:12PM,243871,20,8380693,"ACI Healthcare USA, Inc.",Released
9,Aug 10 2022 12:12PM,243871,20,8380751,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,243872,Released,21
36,243873,Released,1
37,243874,Released,2
38,243876,Released,1
39,243877,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243872,NaN,NaN,21.0
243873,NaN,NaN,1.0
243874,NaN,NaN,2.0
243876,NaN,NaN,1.0
243877,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0.0,0.0,9.0
243778,22.0,6.0,1.0
243783,0.0,1.0,0.0
243795,14.0,6.0,0.0
243805,0.0,0.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243769,0,0,9
243778,22,6,1
243783,0,1,0
243795,14,6,0
243805,0,0,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,0,0,9
1,243778,22,6,1
2,243783,0,1,0
3,243795,14,6,0
4,243805,0,0,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243769,,,9
1,243778,22,6,1
2,243783,,1,
3,243795,14,6,
4,243805,,,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc."
14,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc."
15,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc."
16,Aug 10 2022 11:51AM,243874,19,ACG North America LLC
18,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation
39,Aug 10 2022 11:36AM,243873,19,ACG North America LLC
40,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc."
52,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC"
55,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.
75,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",,,14
1,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc.",,,1
2,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",,,1
3,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,,,2
4,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,,,21
5,Aug 10 2022 11:36AM,243873,19,ACG North America LLC,,,1
6,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc.",,9,3
7,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",,,3
8,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,,13,7
9,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",14,,
1,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc.",1,,
2,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1,,
3,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2,,
4,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21,,
5,Aug 10 2022 11:36AM,243873,19,ACG North America LLC,1,,
6,Aug 10 2022 11:11AM,243832,20,"Exact-Rx, Inc.",3,9,
7,Aug 10 2022 10:22AM,243864,15,"Mizner Bioscience, LLC",3,,
8,Aug 10 2022 10:21AM,243866,10,Eywa Pharma Inc.,7,13,
9,Aug 10 2022 10:21AM,243863,15,"Carwin Pharmaceutical Associates, LLC",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",14,,
1,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc.",1,,
2,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1,,
3,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2,,
4,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",14.0,NaN,NaN
1,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2.0,NaN,NaN
4,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 12:12PM,243871,20,"ACI Healthcare USA, Inc.",14.0,0.0,0.0
1,Aug 10 2022 12:08PM,243877,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Aug 10 2022 12:03PM,243876,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Aug 10 2022 11:51AM,243874,19,ACG North America LLC,2.0,0.0,0.0
4,Aug 10 2022 11:43AM,243872,10,ISDIN Corporation,21.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1219232,40.0,20.0,2.0
12,243783,0.0,1.0,0.0
15,975449,45.0,20.0,0.0
16,243865,1.0,0.0,0.0
19,1706989,7.0,21.0,14.0
20,1950567,78.0,31.0,46.0
21,487658,0.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1219232,40.0,20.0,2.0
1,12,243783,0.0,1.0,0.0
2,15,975449,45.0,20.0,0.0
3,16,243865,1.0,0.0,0.0
4,19,1706989,7.0,21.0,14.0
5,20,1950567,78.0,31.0,46.0
6,21,487658,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,40.0,20.0,2.0
1,12,0.0,1.0,0.0
2,15,45.0,20.0,0.0
3,16,1.0,0.0,0.0
4,19,7.0,21.0,14.0
5,20,78.0,31.0,46.0
6,21,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,12,Released,0.0
2,15,Released,45.0
3,16,Released,1.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,2.0,0.0,0.0,0.0,14.0,46.0,0.0
Executing,20.0,1.0,20.0,0.0,21.0,31.0,2.0
Released,40.0,0.0,45.0,1.0,7.0,78.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,14.0,46.0,0.0
1,Executing,20.0,1.0,20.0,0.0,21.0,31.0,2.0
2,Released,40.0,0.0,45.0,1.0,7.0,78.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,14.0,46.0,0.0
1,Executing,20.0,1.0,20.0,0.0,21.0,31.0,2.0
2,Released,40.0,0.0,45.0,1.0,7.0,78.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()